# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [233]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [232]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [234]:
import textdistance

In [235]:
from tqdm import tqdm

In [236]:
from difflib import get_close_matches

In [239]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [240]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [241]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [237]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [238]:
vocab = Counter(re.findall(r'\w+', corpus.lower()))

Код

In [ ]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    closest = dict(closest)
    p = {}
    # all = []
    for key, val in closest.items():
        p[key] = [val, P(key)]
    res = sorted(p.items(), key=lambda x: x[1][1], reverse=True)   
     
    # for val in p.values():
    #     all.append(val[1])
    #num = max(all)    
    # for k, v in p.items():
    #     if v[1] == num:
    #         return k, v
    # а здесь до меня дошло исп. min и max и я прозрела и поплакала из-за потерянного времени, ведь я все это время смотрела только на цифру перед точкой
    return res[0]

In [479]:
get_closest_hybrid_match("папородник", X, vec)

('папоротник', [0.9, 1.9397592254463824e-06])

In [474]:
get_closest_hybrid_match("бизнез", X, vec)

[('бизнес', [0.8333333333333334, 3.278193091004386e-05]),
 ('бизнеса', [0.7142857142857143, 2.4634942163169054e-05]),
 ('бизнесе', [0.7142857142857143, 5.431325831249871e-06])]

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [242]:
# составляем словарь правильных слов (шаг 1)
correct_sents = open('data/correct_sents.txt', encoding='utf8').read()
vocab_cs = re.findall(r'\w+', correct_sents.lower())
correct_words = dict(zip(vocab_cs, [P(word) for word in vocab_cs])) #с вероятностью
# wiki
vocab = Counter(re.findall(r'\w+', corpus.lower()))
vocab_withp = dict(zip(vocab.keys(), [P(word) for word in vocab.keys()]))

In [243]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [244]:
# удаляем по 1 букве в слове
def get_delete(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return deletes


# составляем словарь удалений (шаг 2)
del_dict = {}
for word in correct_words.keys():
    w = get_delete(word)
    for i in w:
        if i in del_dict.keys():
            del_dict[i].append(word)
        else:
            del_dict[i]=[word]

another_dd = {}
for word in vocab_withp.keys():
    w = get_delete(word)
    for i in w:
        if i in another_dd.keys():
            another_dd[i].append(word)
        else:
            another_dd[i]=[word]

In [245]:
# на вход подается слово с ошибкой, в нем удаляется по букве,
# и если какие-то варианты есть в словаре, они выводятся списком
def del_inc(incorrect_word):
    variants = get_delete(incorrect_word)
    return [i for i in variants if i in del_dict.keys()]

In [246]:
# шаги 3 и 4
def get_correction(incorrect_word):
    correct = del_inc(incorrect_word)
    if len(correct) == 0:
        return incorrect_word
    else:
        # в список добавляются все слова из словаря, в которых содерж. то, что в correct
        words = []
        for i in correct:
            words += del_dict[i]
            words = list(tuple(words))
        # составляем словарь с вариантами и их вероятностями  
        variants = {}
        for i in words:
            variants[i] = correct_words[i]
        result = sorted(variants.items(), key=lambda x:x [1], reverse = True) 
        return result


In [247]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [248]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [249]:
# метрики
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0
mistakes = []
cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab_cs):
            predicted = cashed.get(pair[1], get_correction(pair[1])[0][0])
            cashed[pair[1]] = predicted
        else:
            predicted = pair[1]
        
        if predicted == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], predicted))
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1


100%|██████████| 915/915 [00:00<00:00, 1293.04it/s]


In [250]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8987493746873437
0.2857142857142857
0.01056621109452165


In [254]:
def del_inc2(incorrect_word):
    variants = get_delete(incorrect_word)
    return [i for i in variants if i in another_dd.keys()]

In [255]:
def get_correction2(incorrect_word):
    correct = del_inc2(incorrect_word)
    if len(correct) == 0:
        return incorrect_word
    else:
        # в список добавляются все слова из словаря, в которых содерж. то, что в correct
        words = []
        for i in correct:
            words += another_dd[i]
            words = list(tuple(words))
        # составляем словарь с вариантами и их вероятностями  
        variants = {}
        for i in words:
            variants[i] = vocab_withp[i]
        result = sorted(variants.items(), key=lambda x:x [1], reverse = True) 
        return result

In [257]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0
mistakes = []
cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab_cs):
            predicted = cashed.get(pair[1], get_correction2(pair[1])[0][0])
            cashed[pair[1]] = predicted
        else:
            predicted = pair[1]
        
        if predicted == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], predicted))
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

100%|██████████| 915/915 [00:00<00:00, 933.32it/s] 


In [258]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8890445222611305
0.21040372670807453
0.01056621109452165


In [ ]:
# words = []
# for word in vocab:
#     words.append(word) 

In [ ]:
# dict_delets = {}
# def make_dict(word):
#     deletions = []
#     deletions = (get_delete(word))
#     for delete in deletions:
#         if delete not in dict_delets.keys():
#             dict_delets[delete] = [word]
#         else:
#             dict_delets[delete].append(word)
#     return dict_delets

In [23]:
# make_dict('код')

{'от': ['кот'],
 'кт': ['кот'],
 'ко': ['кот', 'код'],
 'од': ['код'],
 'кд': ['код']}

In [ ]:
# # составляем словарь удалений (шаг 2)
# v = {}
# for word in vocab_withp.keys():
#     w = get_delete(word)
#     for i in w:
#         if i in v.keys():
#             v[i].append(word)
#         else:
#             v[i]=[word]
